<a href="https://colab.research.google.com/github/safdarjung/saf/blob/main/Pytorch_DataLoaders%26Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [92]:
class TabularDataset(Dataset):
  def __init__(self,data,transform=None):
    self.data=data
    self.transform=transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    sample = self.data[idx]
    if self.transform:     #if we apply any transformation
      sample = self.transform(sample)
    return sample



In [93]:
class ToTensor:
  def __call__(self,sample):
    features,labels=sample[0],sample[1]
    return {"features": torch.tensor(features,dtype=torch.float32),"labels":torch.tensor(labels,dtype=torch.float32)}

In [94]:
class Normalize:
  def __call__(self,sample):
    features,labels=sample[0],sample[1]
    norm_features=(features - np.mean(features))/np.std(features)
    return (norm_features,labels)

In [95]:
tabular_data = [(np.random.rand(2),np.random.rand()) for _ in range(100)]

In [96]:
transform = transforms.Compose([Normalize(),ToTensor()])

In [97]:
dataset = TabularDataset(data=tabular_data,transform=transform)

In [98]:
dataloader = DataLoader(dataset,batch_size=16,shuffle=True)

In [99]:
class SimpleNN(nn.Module):
  def __init__(self,input_size):
    super(SimpleNN,self).__init__()
    self.fc1 = nn.Linear(input_size,1)

  def forward(self,x):
    x = self.fc1(x)
    return x

In [100]:
model = SimpleNN(input_size=2)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

In [101]:
num_epochs = 50
for epoch in range(num_epochs):
  model.train()
  t_loss = 0.0

  for batch in dataloader:
    features = batch["features"]
    labels = batch["labels"]
    optimizer.zero_grad()
    outputs = model(features)
    loss = criterion(outputs,labels.view(-1,1))
    loss.backward()
    optimizer.step()

    t_loss += loss.item()

  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {t_loss/len(dataloader)}")

Epoch 1/50, Loss: 1.046856267111642
Epoch 2/50, Loss: 0.8232783419745309
Epoch 3/50, Loss: 0.6477352976799011
Epoch 4/50, Loss: 0.48536755783217295
Epoch 5/50, Loss: 0.38544091156550814
Epoch 6/50, Loss: 0.30276021574224743
Epoch 7/50, Loss: 0.245792778474944
Epoch 8/50, Loss: 0.18918635456689767
Epoch 9/50, Loss: 0.1913736973490034
Epoch 10/50, Loss: 0.15759982062237604
Epoch 11/50, Loss: 0.1372073335306985
Epoch 12/50, Loss: 0.10822382090347153
Epoch 13/50, Loss: 0.1014650437448706
Epoch 14/50, Loss: 0.09852598713976997
Epoch 15/50, Loss: 0.0863503994686263
Epoch 16/50, Loss: 0.08416694828442164
Epoch 17/50, Loss: 0.08713778534105845
Epoch 18/50, Loss: 0.09398863730686051
Epoch 19/50, Loss: 0.07502205084477152
Epoch 20/50, Loss: 0.07765788212418556
Epoch 21/50, Loss: 0.07568262038486344
Epoch 22/50, Loss: 0.0767366960644722
Epoch 23/50, Loss: 0.0825517326593399
Epoch 24/50, Loss: 0.070276551214712
Epoch 25/50, Loss: 0.07826738804578781
Epoch 26/50, Loss: 0.0779667677623885
Epoch 27/5

In [104]:
model.eval()

with torch.no_grad():
  t_loss = 0.0
  for batch in dataloader:
    features = batch["features"]
    labels = batch["labels"]
    outputs = model(features)
    loss = criterion(outputs,labels.view(-1,1))
    t_loss += loss.item()
  print(t_loss/len(dataloader))

0.07373614343149322
